In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import re
import string
import joblib

In [ ]:
#!python -m pip install pandas

     ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.1 MB 1.3 MB/s eta 0:00:09
     ---------------------------------------- 0.1/11.1 MB 1.7 MB/s eta 0:00:07
      --------------------------------------- 0.1/11.1 MB 1.1 MB/s eta 0:00:11
      --------------------------------------- 0.2/11.1 MB 1.1 MB/s eta 0:00:11
      --------------------------------------- 0.2/11.1 MB 1.1 MB/s eta 0:00:11
     - -------------------------------------- 0.3/11.1 MB 1.2 MB/s eta 0:00:10
     - -------------------------------------- 0.4/11.1 MB 1.1 MB/s eta 0:00:10
     - -------------------------------------- 0.4/11.1 MB 1.1 MB/s eta 0:00:10
     - -------------------------------------- 0.5/11.1 MB 1.1 MB/s eta 0:00:10
     - -------------------------------------- 0.5/11.1 MB 1.1 MB/s eta 0:00:10
     -- ------------------------------------- 0.6/11.1 MB 1.1 MB/s eta 0:00:10
     -- ------------------------------------- 0.6/11.1 MB 1


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#!python -m pip install joblib

  Using cached joblib-1.5.1-py3-none-any.whl (307 kB)



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#!python -m pip install scikit-learn

  Using cached scikit_learn-1.7.0-cp310-cp310-win_amd64.whl (10.7 MB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl (41.3 MB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
fake=pd.read_csv('Fake.csv')
true=pd.read_csv('True.csv')

In [6]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [7]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [8]:
fake['class']=0
true['class']=1

In [9]:
data1=pd.concat([fake,true],axis=0)

In [12]:
data1.sample(10)

,text,class
3553,BRUSSELS (Reuters) - U.S. President Donald Tru...,1
2628,When Trump s White House released a list of te...,0
14317,BEIJING (Reuters) - China supports Cambodia s ...,1
14167,CAIRO (Reuters) - Egypt s public prosecutor ha...,1
7652,An Ohio mother who attended a Bernie Sanders r...,0
11240,(Reuters) - Kansas Governor Sam Brownback has ...,1
18861,How long before the media will stop using two ...,0
20718,"PRISTINA (Reuters) - Ramush Haradinaj, a forme...",1
20853,LONDON (Reuters) - Brexit could be reversed if...,1
8661,We all know that the only reason Donald Trump ...,0


In [11]:
data1=data1.drop(['title','subject','date'],axis=1)

In [13]:
def clean_text(text):
    text=text.lower()
    text=re.sub('\[.*?\]',"",text)
    text=re.sub("\\W"," ",text)
    text=re.sub("https?:://\S+|www\.\S+","",text)
    text=re.sub("<.*?>+"," ",text)
    text=re.sub("[%s]"% re.escape(string.punctuation),"",text)
    text=re.sub("\n"," ",text)
    text=re.sub("\w*\d\w*","",text)
    return text

In [14]:
data1['text']=data1['text'].apply(clean_text)

In [15]:
x=data1['text']
y=data1['class']

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=42)

In [16]:
vectorizer = TfidfVectorizer()
xv_train=vectorizer.fit_transform(xtrain)
xv_test=vectorizer.transform(xtest)

In [17]:
lr=LogisticRegression()
lr.fit(xv_train,ytrain)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [18]:
prediction=lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9858351893095768

In [19]:
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [20]:
joblib.dump(vectorizer, 'vectorizer.pkl')
joblib.dump(lr, 'lr_model.pkl')

['lr_model.pkl']